# **Battle of the Languages**
here is our code, blah blah blah, we can write more stuff here later
please feel free to play with the groupings and cells, I just did some initial stuff I thought  might be helpful, but it might need to be broken up more

Importing the Data from the various databases:

https://github.com/festvox/datasets-CMU_Wilderness One of the ones Emily recommended, has like 700 languages, seems like it was mined from people reading the new testament. has polish, spanish, english, (probably has greek but it's labeled by the language in that language so I was not positive what I was looking for)

https://openslr.org/resources.php The other one Emily recommended instead of the UPenn one, had a brief look and seems like it might be more helpful for spanish/english.

In [1]:
#insert code here

Compiling said data to compatible formats and whatnot

In [2]:
#insert code here

Feature Extraction and beyond...